https://datasciencedojo.com/blog/scrape-twitter-data-using-snscrape/

In [44]:
#!pip3 install seaborn

In [45]:
import pandas as pd 
import numpy as np 
import snscrape.modules.twitter as sntwitter 
import datetime 
from tqdm.notebook import tqdm_notebook 
import seaborn as sns 
import matplotlib.pyplot as plt
import re

In [46]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@\S+', '', tweet)
    tweet = re.sub(r'#\S+', '', tweet)
    tweet = remove_emojis (tweet)
    return tweet

In [47]:
def query(since,until): 
    q = "abortion"  
    q += f" until:{until}" 
    q += f" since:{since}"
    #q += f" language: en" 
    return q 

In [48]:
def getTweets(since, until, count):
    q = query(since,until) 
    # Creating list to append tweet data  
    tweets_list1 = [] 
    with tqdm_notebook(total=count) as pbar: 
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()): #declare a username  
                if i>=count: #number of tweets you want to scrape 
                    break 
                if clean_tweet(tweet.rawContent) != "":
                    tweets_list1.append([tweet.date, tweet.id, clean_tweet(tweet.rawContent), tweet.user.username,tweet.lang,tweet.hashtags,tweet.replyCount, tweet.retweetCount,tweet.likeCount,tweet.quoteCount,tweet.media]) 
                    pbar.update(1) 
    # Creating a dataframe from the tweets list above  
    tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username','Language', 'Hashtags','ReplyCount','RetweetCount','LikeCount','QuoteCount','Media']) 
    #tweets_df1.sort_values(by='DateTime',ascending=False) 
    return tweets_df1

In [49]:
since = "2022-08-01" 
until = "2022-08-31"
count = 100 #int(input('Enter max number of tweets or enter -1 to retrieve all possible tweets: ')) 
csv_path = f"{since}_{until}_{count}.csv"
twitterTable = getTweets(since, until, count);
twitterTable.to_csv(csv_path)


  0%|          | 0/100 [00:00<?, ?it/s]